In [1]:
from __future__ import division,print_function,unicode_literals

import numpy as np 
import os 


# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [2]:
import tensorflow as tf

#Declaration before running TensorFlow session
x = tf.Variable(3,name="x")
y = tf.Variable(4,name="y")

f = x * x * y + y + 2

In [3]:
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [4]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


In [5]:
sess.close()

In [6]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [7]:
reset_graph()

x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [8]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2) 
    
x2.graph is graph

True

In [9]:
x2.graph is tf.get_default_graph()

False

In [10]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [11]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


In [12]:
import numpy as np 
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias  = np.c_[np.ones((m, 1)),housing.data]

X = tf.constant(housing_data_plus_bias, dtype=np.float32, name="x")
y = tf.constant(housing.target.reshape(-1, 1),dtype=tf.float32,name="y")
XT = tf.transpose(X)
#Theta = (X^T * X)^-1 * X^T*y 
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [13]:
#Values from TensorFlow
theta_value

array([[-3.74659576e+01],
       [ 4.35208052e-01],
       [ 9.34183039e-03],
       [-1.05619654e-01],
       [ 6.38267040e-01],
       [-4.28281601e-06],
       [-3.77140474e-03],
       [-4.26884502e-01],
       [-4.40567464e-01]], dtype=float32)

In [14]:
#Values from NumPy
X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

print(theta_numpy)

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


In [15]:
#Values from Scikit-Learn
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1,1))

print(np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


In [16]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [17]:
print(scaled_housing_data_plus_bias.mean(axis=0))
print(scaled_housing_data_plus_bias.mean(axis=1))
print(scaled_housing_data_plus_bias.mean())
print(scaled_housing_data_plus_bias.shape)

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ... -0.06612179 -0.06360587
  0.01359031]
0.11111111111111005
(20640, 9)


In [18]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32,name="X")
y = tf.constant(housing.target.reshape(-1, 1),dtype=tf.float32,name="y")
theta = tf.Variable(tf.random_uniform([n +1, 1],-1.0,1.0, seed=42),name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y;
mse = tf.reduce_mean(tf.square(error),name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch",epoch," MSE = ",mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()

Epoch 0  MSE =  9.161543
Epoch 100  MSE =  0.71450055
Epoch 200  MSE =  0.56670463
Epoch 300  MSE =  0.5555716
Epoch 400  MSE =  0.54881155
Epoch 500  MSE =  0.54363626
Epoch 600  MSE =  0.5396292
Epoch 700  MSE =  0.5365092
Epoch 800  MSE =  0.53406775
Epoch 900  MSE =  0.53214705


In [19]:
best_theta

array([[ 2.0685523 ],
       [ 0.8874028 ],
       [ 0.14401658],
       [-0.34770888],
       [ 0.36178374],
       [ 0.00393811],
       [-0.04269556],
       [-0.6614528 ],
       [-0.63752776]], dtype=float32)

In [20]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="X")
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
theta = tf.Variable(tf.random_uniform([n +1, 1],-1.0,1.0,seed=42),name="theta")
y_pred = tf.matmul(X, theta,name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name="mse")i

In [21]:
gradients = tf.gradients(mse, [theta])[0]

In [24]:
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch,"MSE =",mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()
print("Best theta:")
print(best_theta)

Epoch 0 MSE = 9.161543
Epoch 100 MSE = 0.7145006
Epoch 200 MSE = 0.56670463
Epoch 300 MSE = 0.5555715
Epoch 400 MSE = 0.5488117
Epoch 500 MSE = 0.54363626
Epoch 600 MSE = 0.53962916
Epoch 700 MSE = 0.5365091
Epoch 800 MSE = 0.53406775
Epoch 900 MSE = 0.53214705
Best theta:
[[ 2.0685525 ]
 [ 0.8874027 ]
 [ 0.14401652]
 [-0.34770882]
 [ 0.3617837 ]
 [ 0.0039381 ]
 [-0.04269556]
 [-0.6614531 ]
 [-0.637528  ]]


In [25]:
def my_fun(a, b):
    z = 0 
    for i in range(100):
        z = a * np.cos(z + i) + z * np.sin(b - i)
    return z

In [26]:
my_fun(0.2,0.3)

-0.21253923284754914

In [28]:
reset_graph()

a = tf.Variable(0.2, name="a")
b = tf.Variable(0.3, name="b")
z = tf.constant(0.0, name="z0")

for i in range(100):
    z = a * tf.cos(z + i) + z * tf.sin(b - i)
    
grads = tf.gradients(z, [a, b])
init = tf.global_variables_initializer()

In [29]:
with tf.Session() as sess:
    init.run()
    print(z.eval())
    print(sess.run(grads)) 

-0.21253741
[-1.1388494, 0.19671395]


In [30]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=np.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32,name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0, seed=42),name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

In [31]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
tra

In [33]:
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE=",mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()
    
print("Best theta")
print(best_theta)

Epoch 0 MSE= 9.161543
Epoch 100 MSE= 0.7145006
Epoch 200 MSE= 0.56670463
Epoch 300 MSE= 0.5555715
Epoch 400 MSE= 0.5488117
Epoch 500 MSE= 0.54363626
Epoch 600 MSE= 0.53962916
Epoch 700 MSE= 0.5365091
Epoch 800 MSE= 0.53406775
Epoch 900 MSE= 0.53214705
Best theta
[[ 2.0685525 ]
 [ 0.8874027 ]
 [ 0.14401652]
 [-0.34770882]
 [ 0.3617837 ]
 [ 0.0039381 ]
 [-0.04269556]
 [-0.6614531 ]
 [-0.637528  ]]
